In [13]:
from os import path
import pandas as pd
import numpy as np
import re
from datetime import date as dt

In [14]:
CAMPAING = 'AVC_2021_002'
PRE = pd.read_csv(f"./{CAMPAING.replace('_', '-')}.csv")

In [15]:
PRE.columns

Index(['SKU', 'categoria', 'color', 'id_campaign', 'id_product', 'condicion',
       'nombre', 'coleccion', 'foto', 'edicion', 'talla_etiqueta',
       'talla_estimada', 'marca', 'composicion', 'precio', 'descripcion',
       'problema', 'ancho de hombros (AHO)', 'contorno de busto (CBU)',
       'contorno de cintura (CCI)', 'largo de mangas (LMA)',
       'largo total (LTT)', 'contorno de cadera (CCA)', 'largo de tiro (LTI)',
       'largo de planta (LPL)', 'alto de taco (ATA)', 'alto (ALT)',
       'largo (LRG)', 'ancho (ANC)', 'contorno interno (CIN)',
       'diámetro interno (DIN)', 'peso_envio', 'medidas_envio'],
      dtype='object')

In [16]:
PRE = PRE[[
    'SKU', 'categoria', 'color', 'condicion', 'nombre', 'coleccion',
    'talla_etiqueta', 'talla_estimada', 'marca', 'composicion',
    'precio', 'descripcion', 'problema',
    'ancho de hombros (AHO)', 'contorno de busto (CBU)',
    'contorno de cintura (CCI)', 'largo de mangas (LMA)',
    'largo total (LTT)', 'contorno de cadera (CCA)', 'largo de tiro (LTI)',
    'largo de planta (LPL)', 'alto de taco (ATA)', 'alto (ALT)',
    'largo (LRG)', 'ancho (ANC)', 'contorno interno (CIN)', 'diámetro interno (DIN)',
    'peso_envio', 'medidas_envio', 'foto'
]]
PRE.columns = [
    'SKU', 'categoria', 'color', 'condicion', 'nombre', 'coleccion',
    'talla_etiqueta', 'talla_estimada', 'marca', 'composicion',
    'precio', 'descripcion', 'problema',
    'AHO', 'CBU',
    'CCI', 'LMA',
    'LTT', 'CCA', 'LTI',
    'LPL', 'ATA', 'ALT',
    'LRG', 'ANC', 'CIN', 'DIN',
    'peso_envio', 'medidas_envio', 'ready'
]
PRE = PRE.replace(np.nan, '', regex=True)
PRE = PRE[PRE['ready'] == True]

In [17]:
RED_COLS = [
    'Permalink',
    'Name',
    'Description',
    'Meta Title',
    'Meta Description',
    'Width',
    'Length',
    'Height',
    'Brand',
    'Barcode',
    'Categories',
    'Images',
    'Digital',
    'Featured',
    'Status',
    'SKU',
    'Weight',
    'Stock',
    'Stock Unlimited',
    'Price',
    'Custom Field 1 Label',
    'Custom Field 1 Value',
    'Custom Field 1 Type',
    'Custom Field 2 Label',
    'Custom Field 2 Value',
    'Custom Field 2 Type',
    'Custom Field 3 Label',
    'Custom Field 3 Value',
    'Custom Field 3 Type',
    'Google Product Category'
]

CATEGORIES = {
    'ABR': 'Abrigos & Trenchs',
    'BLU': 'Blusas & Poleras',
    'BLZ': 'Chaquetas & Blazers',
    'CHL': 'Chalecos & Polerones',
    'FLD': 'Faldas & Shorts',
    'PNT': 'Pantalones & Jeans',
    'VST': 'Vestidos & Enteritos',
    'LNC': 'Lencería',
    'ZPT': 'Zapatos',
    'ACC': 'Accesorios'
}

GOOGLE_CATEGORIES = {
    'ABR': 'Apparel & Accessories > Clothing > Outerwear > Coats & Jackets',
    'BLU': 'Apparel & Accessories > Clothing > Shirts & Tops',
    'BLZ': 'Apparel & Accessories > Clothing > Outerwear > Coats & Jackets',
    'CHL': 'Apparel & Accessories > Clothing > Outerwear > Chaps',
    'FLD': 'Apparel & Accessories > Clothing > Skirts',
    'PNT': 'Apparel & Accessories > Clothing > Pants',
    'VST': 'Apparel & Accessories > Clothing > Dresses',
    'LNC': 'Apparel & Accessories > Clothing > Underwear & Socks > Lingerie',
    'ZPT': 'Apparel & Accessories > Shoes',
    'ACC': 'Apparel & Accessories > Clothing Accessories'
}

MEASUREMENTS = {
    'AHO': 'ancho de hombros',
    'CBU': 'contorno de busto',
    'CCI': 'contorno de cintura',
    'LMA': 'largo de mangas',
    'LTT': 'largo total',
    'CCA': 'contorno de cadera',
    'LTI': 'largo de tiro',
    'LPL': 'largo de planta',
    'ATA': 'alto de taco',
    'ALT': 'alto',
    'LRG': 'largo',
    'ANC': 'ancho',
    'CIN': 'contorno interno',
    'DIN': 'diámetro interno'
}

COLORS = {
    'AMR': 'amarillo',
    'AZL': 'azul',
    'BGE': 'beige',
    'BLN': 'blanco',
    'BYN': 'blanco & negro',
    'CFE': 'café',
    'CLP': 'calipso',
    'CLT': 'celeste',
    'COB': 'cobre',
    'DOR': 'dorado',
    'GRS': 'gris',
    'MLT': 'multicolor',
    'MRD': 'morado',
    'NGR': 'negro',
    'NRJ': 'naranjo',
    'NVY': 'navy',
    'PLA': 'plateado',
    'RJO': 'rojo',
    'RSD': 'rosado',
    'TRN': 'transparente',
    'VRD': 'verde'
}

def compose_description(df):
    # description
    description = df['descripcion'].strip().split()
    description_problems = df['problema'].strip().split()
    description_t = f"<p><b>Descripción:</b> {' '.join(description)} {' '.join(description_problems)}</p>"
    # brand
    brand = df['marca'].strip()
    brand_t = f"<p><b>Marca:</b> {brand}</p>" if brand else ""
    # estimated size
    est_size = df['talla_estimada'].strip()
    est_size_t = f'<p><b>Talla estimada:</b> {est_size}</p>' if est_size else ''# if est_size in ['XS', 'S', 'SM', 'M', 'ML'. 'L', 'XL'] else f'<p><b>Talla estimada:</b> {est_size}/{get_size(est_size)}</p>'
    # label size
    lab_size = df['talla_etiqueta'].strip()
    lab_size_t = f'<p><b>Talla etiqueta:</b> {lab_size}</p>' if lab_size else ''
    # composition
    composition = [ i.split("/") for i in df['composicion'].split('//')]
    composition_d = {}
    if composition != [[""]]:
        composition_d['e'] = [re.sub("\d+", lambda ele: ele[0] + "%", i.strip()) for i in composition[0]]
        if len(composition) == 2:
            composition_d['i'] = [re.sub("\d+", lambda ele: ele[0] + "%", i.strip()) for i in composition[1]]
        else:
            composition_d['i'] = False

        composition_t = f"<p><b>Composición:</b> {' / '.join(composition_d['e'])}</p>"
        composition_t += f"<p><b>Composición del forro:</b> {' / '.join(composition_d['i'])}</p>" if composition_d['i'] else ""
    else:
        composition_t = ""
    # measurements
    measurements = [f"{MEASUREMENTS[i]}: {int(df[i])} {'cm' if i != 'DIN' else 'mm'}" for i in ['AHO', 'CBU', 'CCI', 'LMA', 'LTT', 'CCA', 'LTI', 'LPL', 'ATA', 'ALT', 'LRG', 'CIN', 'DIN'] if df[i]]
    if measurements != []:
        measurements_t = f"<p><b>Medidas:</b> {' / '.join(measurements)}</p>"
    else:
        measurements_t = ""
    # whole description
    return f'{description_t}{brand_t}{est_size_t}{lab_size_t}{measurements_t}{composition_t}'

def compose_categories(df):
    collection = df["coleccion"]
    collection_code = f'Colecciones,Colecciones / {collection},' if collection else ''
    category = CATEGORIES[df["categoria"]]
    category_code = f'Productos,Productos / {category}'
    return f'{collection_code}{category_code}'

def compose_images(df):
    sku = df["SKU"]
    base_url = "https://raw.githubusercontent.com/RamonOpazo/AmigasVntg_Catalog/master"
    base_dir = CAMPAING
    imgs = [f'{sku}-{i}.jpg' for i in range(1,10)]
    imgs = [f'{str(path.join(base_url, base_dir, i))}' for i in imgs]
    imgs_url = ",".join(imgs)
    return imgs_url

def get_color(col):
    return COLORS[col]

def get_size(col):
    if col in ['XS', 'S', 'SM', 'M', 'ML', 'L', 'XL']:
        return col
    else:
        col = int(col)
        if col <= 34:
            return 'XS'
        elif col <= 38:
            return 'S'
        elif col <= 42:
            return 'M'
        elif col <= 46:
            return 'L'
        elif col > 46:
            return 'XL'
        
def get_google_category(col):
    return GOOGLE_CATEGORIES[col]

def get_corrected_weight(col):
    n1 = int(col*1.1)
    n2 = n1 + (5 - n1 % 5)
    n3 = int(n2)/1000
    return n3

def compose_package_size(data):
    if data:
        w, l, h = [ int(i) for i in data.split(',')]
    else:
        w, h, l = 10, 10, 10
    return {"w": w, "l": l, "h": h}

def compose_package_w(data):
    return compose_package_size(data)["w"]

def compose_package_l(data):
    return compose_package_size(data)["l"]

def compose_package_h(data):
    return compose_package_size(data)["h"]

def compose_price(data):
    price = int(data.replace("$", "").replace(".", ""))
    return price

In [18]:
RED = pd.DataFrame(columns=RED_COLS)
RED['Name'] = PRE['nombre'] #PRE['SKU'].apply(lambda x: x[0:7]) #PRE['nombre'] #PRE['SKU'].apply(lambda x: x[0:7]) # Needs real name
RED['Description'] = PRE.apply(compose_description, axis=1)
RED['Meta Description'] = PRE['descripcion']
RED['SKU'] = PRE['SKU']
RED['Width'] = PRE['medidas_envio'].apply(compose_package_w)
RED['Length'] = PRE['medidas_envio'].apply(compose_package_l)
RED['Height'] = PRE['medidas_envio'].apply(compose_package_h)
RED['Brand'] = PRE['marca']
RED['Categories'] = PRE.apply(compose_categories, axis=1)
RED['Images'] = PRE.apply(compose_images, axis=1)
RED[['Digital', 'Featured', 'Stock Unlimited']] = 'NO'
RED['Status'] = ['available', 'not-available', 'disabled'][2]
RED['Weight'] = PRE['peso_envio'].apply(get_corrected_weight) #PRE['peso_envio']/1000
RED['Stock'] = 1
RED['Price'] = PRE['precio'].apply(compose_price)
#RED['Price'] = PRE['precio'].apply(lambda x: int(x.replace("$", "").replace(".", "")))
RED['Custom Field 1 Label'] = 'color'
RED['Custom Field 1 Value'] = PRE['color'].apply(get_color)
RED['Custom Field 1 Type'] = 'selection'
RED['Custom Field 2 Label'] = 'talla'
RED['Custom Field 2 Value'] = PRE['talla_estimada']#.apply(get_size)
RED['Custom Field 2 Type'] = 'selection'
RED['Custom Field 3 Label'] = 'condición'
RED['Custom Field 3 Value'] = PRE['condicion']
RED['Custom Field 3 Type'] = 'selection'
RED['Google Product Category'] = PRE['categoria'].apply(get_google_category)
RED[['Permalink', 'Meta Title', 'Barcode']] = ""

In [19]:
RED

,Permalink,Name,Description,Meta Title,Meta Description,Width,Length,Height,Brand,Barcode,...,Custom Field 1 Label,Custom Field 1 Value,Custom Field 1 Type,Custom Field 2 Label,Custom Field 2 Value,Custom Field 2 Type,Custom Field 3 Label,Custom Field 3 Value,Custom Field 3 Type,Google Product Category
0,,Bufanda de lana café,<p><b>Descripción:</b> Abrigada bufanda de lan...,,"Abrigada bufanda de lana, con diseño geométric...",10,10,10,,,...,color,café,selection,talla,,selection,condición,usado,selection,Apparel & Accessories > Clothing Accessories
1,,Pantalón de cotelé beige,<p><b>Descripción:</b> Pantalón de cotelé suav...,,"Pantalón de cotelé suave y abrigado, de color ...",10,10,10,Carrera,,...,color,beige,selection,talla,40,selection,condición,usado,selection,Apparel & Accessories > Clothing > Pants
2,,Jeans negros Ooh La La,<p><b>Descripción:</b> Jeans de mezclilla negr...,,"Jeans de mezclilla negra, tiro medio y clásico...",10,10,10,Ooh La La,,...,color,negro,selection,talla,40,selection,condición,usado,selection,Apparel & Accessories > Clothing > Pants
3,,Pantalón de cotelé negro,<p><b>Descripción:</b> Clásico pantalón de cot...,,Clásico pantalón de cotelé negro y corte recto...,10,10,10,Holiday,,...,color,negro,selection,talla,42,selection,condición,usado,selection,Apparel & Accessories > Clothing > Pants
4,,Pantalón de cotelé cherry,<p><b>Descripción:</b> Alegre pantalón de cote...,,Alegre pantalón de cotelé rojo con cierre y br...,10,10,10,Colmar,,...,color,rojo,selection,talla,38,selection,condición,usado,selection,Apparel & Accessories > Clothing > Pants
5,,Jeans clásico Levi's,<p><b>Descripción:</b> Perfecto y clásico blue...,,Perfecto y clásico blue jeans Levi's de corte ...,10,10,10,Levi,,...,color,azul,selection,talla,38,selection,condición,usado,selection,Apparel & Accessories > Clothing > Pants
6,,Vestido negro blossom,<p><b>Descripción:</b> Hermoso vestido negro c...,,"Hermoso vestido negro con estampado floral, de...",10,10,10,,,...,color,negro,selection,talla,L,selection,condición,usado,selection,Apparel & Accessories > Clothing > Dresses
7,,Vestido black & white,<p><b>Descripción:</b> Elegante vestido ochent...,,Elegante vestido ochentero en blanco y negro. ...,10,10,10,Demaro,,...,color,blanco & negro,selection,talla,S,selection,condición,usado,selection,Apparel & Accessories > Clothing > Dresses
8,,Vestido black forest,<p><b>Descripción:</b> Vestido negro con delic...,,"Vestido negro con delicado bordado floral, tam...",10,10,10,Yessica,,...,color,negro,selection,talla,42,selection,condición,usado,selection,Apparel & Accessories > Clothing > Dresses
9,,Vestido blue phantasy,<p><b>Descripción:</b> Hermoso vestido de tela...,,Hermoso vestido de tela plisada y acabado metá...,10,10,10,C&A,,...,color,azul,selection,talla,44,selection,condición,usado,selection,Apparel & Accessories > Clothing > Dresses


In [20]:
today = dt.today()
today = today.strftime("%Y%m%d")
RED.to_csv(f'./{today}_READY.csv', index=False)